In [16]:
import pandas as pd
import logging
import boto3
from botocore.exceptions import ClientError
import numpy as np 
import kaggle
from subprocess import call

In [27]:
#we run the script where we have written dowload of the kaggle dataset by using kaggle api
#the file will be downloaded to the current folder where you have this script 
rc = call("./script.sh", shell=True)
print("download successful")

In [19]:
#we are writing a fuction to help us load the file to aws s3 bucket 
def upload_file(file_name, bucket, object_name=None):
    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    print("uploaded {} to bucket {} ".format(object_name, bucket))

In [20]:
#here the function converts the file to parquet and uploads to the s3 bucket 
def convert_to_parquet(dataset):
    for i in range(len(dataset)):
        dataset[i].to_parquet(f'data/ParisHousing_period_{i + 1:02d}.parquet')
        
        file_name = f"/home/deen/Desktop/datatalks/Mlops-project/dataset/python_script/data/ParisHousing_period_{i + 1:02d}.parquet"
        bucket = "mlops-project-dataset-deen"
        object_name = f"paris-housing-dataset/ParisHousing_period_{i + 1:02d}.parquet"
        upload_file(file_name, bucket, object_name)

In [22]:
#we state the location the of the downloaded file, sort it by date and reset the indexes 
path = "./ParisHousingClass.csv"
df = pd.read_csv(path)
df.sort_values(by=['made'], inplace=True)
df = df.reset_index()


In [23]:
#here we split the file into four equal parts, so that we can act like the file comes at the end of every period
np.split(df,4)
period_01 = np.split(df,4)[0]
period_02 = np.split(df,4)[1]
period_03 = np.split(df,4)[2]
period_04 = np.split(df,4)[3]

In [24]:
#we create inset the tables into an array called data 
data = [period_01,period_02,period_03,period_04]

In [25]:
#we create a folder where we want the splitted data to enter
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [26]:
#we call the fuction "convert to parquet that we created above "
convert_to_parquet(data)

uploaded paris-housing-dataset/ParisHousing_period_01.parquet to bucket mlops-project-dataset-deen 
uploaded paris-housing-dataset/ParisHousing_period_02.parquet to bucket mlops-project-dataset-deen 
uploaded paris-housing-dataset/ParisHousing_period_03.parquet to bucket mlops-project-dataset-deen 
uploaded paris-housing-dataset/ParisHousing_period_04.parquet to bucket mlops-project-dataset-deen 
